In [54]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import rc, rcParams

warnings.filterwarnings('ignore')
rc('font', family='AppleGothic')
rcParams['axes.unicode_minus']= False

In [55]:

train = pd.read_csv("../2nd_team/data/train_new.csv")
test = pd.read_csv("../2nd_team/data/test_new.csv")
sub = pd.read_csv('../2nd_team/data/sample_submission.csv')
age = pd.read_csv("../2nd_team/data/age_gender_info.csv")

train.shape, test.shape, sub.shape, age.shape

((3184, 61), (776, 61), (150, 2), (16, 23))

### load library

In [56]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import PolynomialFeatures

from sklearn.preprocessing import StandardScaler

[용어정리]  EDA 결과 임대보증금/임대료와 관련있을 것 같은 피처를 `critial_feature`라고 부르자  
- '지역_lb', '총세대수', '버스정류장', '단지내주차면수', '전용면적', '공급유형_lb','자격유형_lb'   
- 이 때 _lb 된 피처는 원핫인코딩을 사용하는 것이 좋다  

### 원핫인코딩을 한 feature와 아닌 feature 분류
* 수치형 feature는 sel_num >>  sel_num = ['총세대수', '버스정류장', '단지내주차면수', '전용면적']
* 원핫인코딩을 한 feature >> '지역_lb', '공급유형_lb','자격유형_lb' 

### Ridge_임대보증금_MinMaxScaler()

In [57]:
sel = ['총세대수', '버스정류장', '단지내주차면수', '전용면적', 
       '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', 
       '서울특별시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도',
       '공공분양', '공공임대(10년)', '공공임대(50년)', '공공임대(5년)',
       '공공임대(분납)', '국민임대', '영구임대', '임대상가', '장기전세', '행복주택',
       'A', 'B','C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']
sel_num = ['총세대수', '버스정류장', '단지내주차면수', '전용면적']
X = train[sel]
y = train[['임대보증금']]

In [58]:
# 피처스케일링
scaler = MinMaxScaler() #  StandardScaler
X_nor = scaler.fit_transform(X)   #['총세대수', '버스정류장', '단지내주차면수', '전용면적' ]을 의미
print("기존 X_nor의 shape : ", X_nor.shape)
tmp_X_nor = X_nor[:,:4] #['총세대수', '버스정류장', '단지내주차면수', '전용면적' ]을 의미
# print(tmp_X_nor)

기존 X_nor의 shape :  (3184, 45)


* 수치형 feature는 polynomial으로 

In [59]:
# polynomial features
X_poly = PolynomialFeatures(degree=2, include_bias=False).fit_transform(tmp_X_nor)  
print("수치형 피처 polynomial X_nor의 shape : ", X_poly.shape)
# print(X_poly[:,:4])

수치형 피처 polynomial X_nor의 shape :  (3184, 14)


In [60]:
X_all = np.concatenate((X_poly, X_nor[:,4:]), axis=1)
print('수치형 피처 polynomial + 기존 원핫인코딩 피처의 shape', X_all.shape)

수치형 피처 polynomial + 기존 원핫인코딩 피처의 shape (3184, 55)


In [61]:
# one-hot-encoding
knn = KNeighborsRegressor()
rf = RandomForestRegressor()

In [62]:
# 회귀는 교차검증
knn_scores = cross_val_score(knn, X_all, y,scoring='neg_mean_absolute_error', cv=5)
rf_scores = cross_val_score(rf, X_all, y,scoring='neg_mean_absolute_error', cv=5)

In [63]:
knn_score = np.abs(knn_scores.mean())
rf_score = np.abs(rf_scores.mean())

In [64]:
print("KNeighborsRegressor : ", knn_score)
print("RandomForestRegressor : ", rf_score)

KNeighborsRegressor :  6433068.1036501685
RandomForestRegressor :  6101082.369316808


### # Ridge, Lasso

In [65]:
from sklearn.linear_model import Lasso, Ridge

In [66]:
alphas = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1]


# for a in alphas:
# ridge = Ridge(alpha=a).fit(X,y)
# score = ridge.score(X, y)


# print("Alpha:{0:.4f}.format(a, score)


In [67]:
# Ridge, Lasso
lasso = Lasso()
ridge = Ridge(alpha=0.1)
lasso_scores = cross_val_score(lasso, X_all, y,scoring='neg_mean_absolute_error', cv=5)
ridge_scores = cross_val_score(ridge, X_all, y,scoring='neg_mean_absolute_error', cv=5)

In [68]:
lasso_score = np.abs(lasso_scores.mean()) #np.abs :절대값
ridge_score = np.abs(ridge_scores.mean())

In [69]:
print("Lasso (alpha=defalt) : ", lasso_score)
print("Ridge (alpha=defalt) : ", ridge_score)

Lasso (alpha=defalt) :  5948604.232416377
Ridge (alpha=defalt) :  5881872.139049062


### Ridge_임대료_MinMaxScaler()

In [70]:
sel = ['총세대수', '버스정류장', '단지내주차면수', '전용면적', 
       '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', 
       '서울특별시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도',
       '공공분양', '공공임대(10년)', '공공임대(50년)', '공공임대(5년)',
       '공공임대(분납)', '국민임대', '영구임대', '임대상가', '장기전세', '행복주택',
       'A', 'B','C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']
sel_num = ['총세대수', '버스정류장', '단지내주차면수', '전용면적']
X = train[sel]
y = train[['임대료']]

In [71]:
# 피처스케일링
scaler = MinMaxScaler() #  StandardScaler
X_nor = scaler.fit_transform(X)
print("기존 X_nor의 shape : ", X_nor.shape)
tmp_X_nor = X_nor[:,:4]
# print(tmp_X_nor)


기존 X_nor의 shape :  (3184, 45)


In [72]:
# polynomial features
X_poly = PolynomialFeatures(degree=2, include_bias=False).fit_transform(tmp_X_nor)  
print("수치형 피처 polynomial X_nor의 shape : ", X_poly.shape)
# print(X_poly[:,:4])

수치형 피처 polynomial X_nor의 shape :  (3184, 14)


In [73]:
X_all = np.concatenate((X_poly, X_nor[:,4:]), axis=1)
print('수치형 피처 polynomial + 기존 원핫인코딩 피처의 shape', X_all.shape)

수치형 피처 polynomial + 기존 원핫인코딩 피처의 shape (3184, 55)


In [74]:
# one-hot-encoding
knn = KNeighborsRegressor()
rf = RandomForestRegressor()

In [75]:
# 회귀는 교차검증
knn_scores = cross_val_score(knn, X_all, y,scoring='neg_mean_absolute_error', cv=5)
rf_scores = cross_val_score(rf, X_all, y,scoring='neg_mean_absolute_error', cv=5)

In [76]:
knn_score = np.abs(knn_scores.mean())
rf_score = np.abs(rf_scores.mean())

In [77]:
print("KNeighborsRegressor : ", knn_score)
print("RandomForestRegressor : ", rf_score)

KNeighborsRegressor :  42289.03919611386
RandomForestRegressor :  40115.46702910905


In [78]:
# Ridge, Lasso
lasso = Lasso()
ridge = Ridge(alpha=10).fit(X, y) # 이 코드 질문
lasso_scores = cross_val_score(lasso, X_all, y,scoring='neg_mean_absolute_error', cv=5)
ridge_scores = cross_val_score(ridge, X_all, y,scoring='neg_mean_absolute_error', cv=5)

In [79]:
lasso_score = np.abs(lasso_scores.mean())
ridge_score = np.abs(ridge_scores.mean())

In [81]:
print("Lasso (alpha=defalt) : ", lasso_score)
print("Ridge (alpha=10) : ", ridge_score)

Lasso (alpha=defalt) :  42054.55005895971
Ridge (alpha=10) :  40032.17990627226


### Ridge_임대료_StandardScaler()

In [42]:
sel = ['총세대수', '버스정류장', '단지내주차면수', '전용면적', 
       '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', 
       '서울특별시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도',
       '공공분양', '공공임대(10년)', '공공임대(50년)', '공공임대(5년)',
       '공공임대(분납)', '국민임대', '영구임대', '임대상가', '장기전세', '행복주택',
       'A', 'B','C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']
sel_num = ['총세대수', '버스정류장', '단지내주차면수', '전용면적']
X = train[sel]
y = train[['임대료']]

In [43]:
# 피처스케일링
scaler = StandardScaler()  # MinMaxScaler()
X_nor = scaler.fit_transform(X)
print("기존 X_nor의 shape : ", X_nor.shape)
tmp_X_nor = X_nor[:,:4]
# print(tmp_X_nor)


기존 X_nor의 shape :  (3184, 45)


In [44]:
# polynomial features
X_poly = PolynomialFeatures(degree=2, include_bias=False).fit_transform(tmp_X_nor)  
print("수치형 피처 polynomial X_nor의 shape : ", X_poly.shape)
# print(X_poly[:,:4])

수치형 피처 polynomial X_nor의 shape :  (3184, 14)


In [45]:
X_all = np.concatenate((X_poly, X_nor[:,4:]), axis=1)
print('수치형 피처 polynomial + 기존 원핫인코딩 피처의 shape', X_all.shape)

수치형 피처 polynomial + 기존 원핫인코딩 피처의 shape (3184, 55)


In [46]:
# one-hot-encoding
knn = KNeighborsRegressor()
rf = RandomForestRegressor()

In [47]:
# 회귀는 교차검증
knn_scores = cross_val_score(knn, X_all, y,scoring='neg_mean_absolute_error', cv=5)
rf_scores = cross_val_score(rf, X_all, y,scoring='neg_mean_absolute_error', cv=5)

In [48]:
knn_score = np.abs(knn_scores.mean())
rf_score = np.abs(rf_scores.mean())

In [49]:
print("KNeighborsRegressor : ", knn_score)
print("RandomForestRegressor : ", rf_score)

KNeighborsRegressor :  47633.681204150744
RandomForestRegressor :  40356.37432583452


In [50]:
# Ridge, Lasso
lasso = Lasso()
ridge = Ridge(alpha=0.1).fit(X, y) # 이 코드 질문
lasso_scores = cross_val_score(lasso, X_all, y,scoring='neg_mean_absolute_error', cv=5)
ridge_scores = cross_val_score(ridge, X_all, y,scoring='neg_mean_absolute_error', cv=5)

In [51]:
lasso_score = np.abs(lasso_scores.mean())
ridge_score = np.abs(ridge_scores.mean())

In [53]:
print("Lasso (alpha=defalt) : ", lasso_score)
print("Ridge (alpha=0.1) : ", ridge_score)

Lasso (alpha=defalt) :  42296.160875544614
Ridge (alpha=0.1) :  42263.673844827965
